In [1]:
import os
import pyodbc
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
def get_db_connection(DB_NAME:str):
    conn = pyodbc.connect(
        f"DRIVER={os.getenv('DB_DRIVER')};"
        f"SERVER={os.getenv('DB_SERVER')};"
        f"DATABASE={DB_NAME};"
        f"UID={os.getenv('DB_UID')};"
        f"PWD={os.getenv('DB_PWD')};"
        f"TrustServerCertificate={os.getenv('TRUST_SERVER_CERTIFICATE')};"
    )
    
    return conn


Save to sql server

In [4]:
account_df = pd.read_csv('./data/account.csv')
customer_df = pd.read_csv('./data/customer.csv')
manager_df = pd.read_csv('./data/manager.csv')

DB_NAME = os.getenv('DB_NAME')
conn = get_db_connection(DB_NAME)
cursor = conn.cursor()

In [5]:
cursor.execute("""
    IF OBJECT_ID('customer', 'U') IS NOT NULL DROP TABLE customer;
               
    IF OBJECT_ID('manager', 'U') IS NOT NULL DROP TABLE manager               

    IF OBJECT_ID('account', 'U') IS NOT NULL DROP TABLE account;
""")

cursor.execute("""
    CREATE TABLE account (
        id INT PRIMARY KEY IDENTITY(1,1),
        username NVARCHAR(255) NOT NULL,
        password NVARCHAR(255) NOT NULL,
        full_name NVARCHAR(255) NOT NULL,
        email NVARCHAR(255) UNIQUE NOT NULL,
        status NVARCHAR(50) DEFAULT 'Active',
        created_at DATETIME DEFAULT GETDATE(),
        updated_at DATETIME
    )
""")

cursor.execute("""
    CREATE TABLE customer (
        id INT PRIMARY KEY IDENTITY(1,1),
        account_id INT NOT NULL,
        phone_number NVARCHAR(15),
        address NVARCHAR(500),
        CONSTRAINT fk_customer_account FOREIGN KEY (account_id) REFERENCES account(id)
    )
""")

cursor.execute("""
    CREATE TABLE manager (
        id INT PRIMARY KEY IDENTITY(1,1),
        role_id INT NOT NULL,
        account_id INT NOT NULL,
        CONSTRAINT fk_manager_role FOREIGN KEY (role_id) REFERENCES role(id),
        CONSTRAINT fk_manager_account FOREIGN KEY (account_id) REFERENCES account(id)
    )
""")

In [6]:
account_tuples = [
    (index, row['username'], row['password'], row['name'], row['email'], row['status'], row['created_at'], row['updated_at'])
    for index, row in tqdm(account_df.iterrows(), desc="Creating account tuples", total=account_df.shape[0], unit="row")
]

customer_tuples = [
    (index, row['account_id'], row['phone_number'], row['address'])
    for index, row in tqdm(customer_df.iterrows(), desc="Creating customer tuples", total=customer_df.shape[0], unit="row")
]

manager_tuples = [
    (index, int(row['role_id']), int(row['account_id']))
    for index, row in tqdm(manager_df.iterrows(), desc="Creating manager tuples", total=manager_df.shape[0], unit="row")
]  

Creating manager tuples: 100%|██████████| 200/200 [00:00<00:00, 13486.29row/s]


In [7]:
cursor.execute("SET IDENTITY_INSERT account ON")
cursor.executemany(
    "INSERT INTO account (id, username, password, full_name, email, status, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
    account_tuples
)
cursor.execute("SET IDENTITY_INSERT account OFF")

In [8]:
cursor.execute("SET IDENTITY_INSERT customer ON")
cursor.executemany(
    "INSERT INTO customer (id, account_id, phone_number, address) VALUES (?, ?, ?, ?)",
    customer_tuples
)
cursor.execute("SET IDENTITY_INSERT customer OFF")

In [9]:
cursor.execute("SET IDENTITY_INSERT manager ON")
cursor.executemany(
    "INSERT INTO manager (id, role_id, account_id) VALUES (?, ?, ?)",
    manager_tuples
)
cursor.execute("SET IDENTITY_INSERT manager OFF")

In [10]:
conn.commit()
conn.close()